# Standard packages

In [ ]:
import os
import pandas as pd
import time

# Specific installs

In [ ]:
method_name = "RNAstrcuture"

In [ ]:
!wget -q http://rna.urmc.rochester.edu/Releases/current/RNAstructureLinuxTextInterfaces64bit.tgz
!tar xfz RNAstructureLinuxTextInterfaces64bit.tgz

In [ ]:
source_path = "/content/RNAstructure/"
install_path = "/usr/local/RNAstructure/"

%cd /content/RNAstructure
!make install
%cd /content

/content/RNAstructure
PY: # ,  
cp -r exe/ /usr/local/RNAstructure
/content


# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [ ]:
import pandas as pd 
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [ ]:
# Sequences to process
sce18 = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
         "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
         "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [ ]:
def run_folding(fasta_name):

  tempK = 273.15 + 25

  # Compute structure
  os.system("export DATAPATH=" + source_path + "data_tables; " + 
            install_path + "Fold " +
            "--bracket --MFE " + "-t " + str(tempK) + " " +
            fasta_name + " tmp.dot")
  
  # Clean output
  out_file_name = "clean_tmp.dot"
  in_lines = open("tmp.dot","r").readlines()
  with open(out_file_name,"w") as out_file:
    for line in in_lines:
      if line[0]==">": # extract just the ID
        sline = line.split(' ')
        out_file.write(">" + sline[4])
      else:
        out_file.write(line)

  return out_file_name

In [ ]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

print("   \t lnc \t len \t time")
for i, lnc in enumerate(sce18): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(sce18)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 0.3 s
2/18	snR34 	 203	 0.3 s
3/18	snR43 	 209	 0.3 s
4/18	snR44 	 211	 0.3 s
5/18	snR31 	 225	 0.4 s
6/18	snR10 	 245	 0.4 s
7/18	snR63 	 255	 0.5 s
8/18	snR11 	 258	 0.5 s
9/18	snR82 	 268	 0.5 s
10/18	snR17b 	 332	 0.8 s
11/18	snR17a 	 333	 0.7 s
12/18	snR37 	 386	 1.1 s
13/18	SCR1 	 522	 2.5 s
14/18	SRG1 	 551	 2.4 s
15/18	snR19 	 568	 3.1 s
16/18	snR30 	 606	 3.6 s
17/18	LSR1 	 1175	 24.5 s
18/18	TLC1 	 1301	 29.0 s
